<a href="https://colab.research.google.com/github/geeekgod/-git-github-workshop-contribution/blob/main/TTS_Hindi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchcodec>=0.6.1
!pip install torch torchaudio
!apt-get update -y && apt-get install -y ffmpeg

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [80.2 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,783 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,305 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Pa

In [2]:
import torch, torchcodec, sys
print(sys.version)          # should show 3.12
print(torch.__version__)    # 2.3.0.dev…  (nightly)
print(torchcodec.__version__)  # ≥0.6.1

3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
2.8.0+cu126
0.6.0


In [3]:
from google.colab import userdata
from huggingface_hub import login

# Loading HF token from Colab Secrets
hf_token = userdata.get('HF_TOKEN')

In [4]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/IndicVoices", "hindi",
                       split="valid", streaming=True)

README.md:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/91 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

In [5]:
print(next(iter(dataset)))

{'audio_filepath': <datasets.features._torchcodec.AudioDecoder object at 0x7baff5fb1c40>, 'text': 'हाँ कौन', 'duration': 1.409, 'lang': 'hi', 'samples': 22544, 'verbatim': 'हाँ कौन', 'normalized': 'हाँ कौन', 'speaker_id': 'S4258409000351307', 'scenario': 'Conversation', 'task_name': 'Conversation', 'gender': 'Female', 'age_group': '45-60', 'job_type': 'White Collar', 'qualification': 'Post Grad + PhD', 'area': 'Rural', 'district': 'Samastipur', 'state': 'Bihar', 'occupation': 'Farming', 'verification_report': "{'decision': 'excellent', 'low_volume': False, 'noise_intermittent': False, 'chatter_intermittent': False, 'noise_persistent': False, 'chatter_persistent': False, 'unclear_audio': False, 'off_topic': False, 'repeating_content': False, 'long_pauses': False, 'mispronunciation': False, 'reading_prompt': False, 'book_read': False, 'sst': False, 'stretching': False, 'bad_extempore_quality': False, 'comments': 'excellent', 'objectionable_content': False, 'skipping_words': False, 'incor

In [6]:
# Additional VITS-specific dependencies
!pip install datasets transformers huggingface_hub
!pip install monotonic-align cython numpy scipy
!pip install tensorboard librosa soundfile
!pip install flask flask-socketio websockets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for monotonic-align (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for monotonic-align
Failed to build monotonic-align
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (monotonic-align)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.7 MB/s eta 0:00:00


In [7]:
# Clone the official VITS repository
!git clone https://github.com/jaywalnut310/vits.git
%cd vits

# Build Monotonic Alignment Search (critical for VITS)
%cd monotonic_align
!python setup.py build_ext --inplace
%cd ../

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 7.04 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/vits
/content/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returne

In [11]:
import traceback
from itertools import islice
from datasets import load_dataset
from google.colab import userdata
from huggingface_hub import login

def debug_audio_decoder():
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    ds = load_dataset("ai4bharat/IndicVoices", "hindi", split="train", streaming=True)
    print("🔍 Inspecting AudioDecoder methods on first sample…")
    for i, sample in enumerate(islice(ds, 1)):
        decoder = sample["audio_filepath"]
        print(f"Type: {type(decoder)}")
        print("Available attributes/methods:", [m for m in dir(decoder) if not m.startswith("_")])
        try:
            if callable(decoder):
                res = decoder()
                print("Called directly →", type(res),
                      "keys" if isinstance(res, dict) else "",
                      list(res.keys()) if isinstance(res, dict) else "")
        except Exception as e:
            print("Direct call failed:", e)
        try:
            if hasattr(decoder, "decode"):
                res = decoder.decode()
                print("decode() →", type(res),
                      "keys" if isinstance(res, dict) else "",
                      list(res.keys()) if isinstance(res, dict) else "")
        except Exception as e:
            print("decode() failed:", e)
        try:
            if hasattr(decoder, "array"):
                arr = decoder.array
                print("array attr →", type(arr),
                      getattr(arr, "shape", "no shape"))
        except Exception as e:
            print("array attr failed:", e)
        try:
            if hasattr(decoder, "sampling_rate"):
                print("sampling_rate attr →", decoder.sampling_rate)
        except Exception as e:
            print("sampling_rate failed:", e)
    print("🔍 Debug complete.")

debug_audio_decoder()


Resolving data files:   0%|          | 0/91 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

🔍 Inspecting AudioDecoder methods on first sample…
Type: <class 'datasets.features._torchcodec.AudioDecoder'>
Available attributes/methods: ['get_all_samples', 'get_samples_played_in_range', 'metadata', 'stream_index']
🔍 Debug complete.


In [16]:
import os
import json
import torch
import torchaudio
from pathlib import Path
from itertools import islice
import traceback
import numpy as np


# Configuration
NUM_OF_SAMPLES = 100
SAMPLING_RATE_TARGET = 22050


def prepare_vits_dataset_fixed(
    dataset,
    output_dir="./hindi_vits_data",
    max_samples=NUM_OF_SAMPLES,
    target_sr=SAMPLING_RATE_TARGET,
    min_text_length=3,  # Lowered from 5 to allow shorter texts
    max_text_length=200
):
    """
    Fixed version for AudioDecoder processing using get_all_samples()
    """
    print(f"🎵 Processing up to {max_samples} samples...")

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f"{output_dir}/wavs", exist_ok=True)

    metadata = []
    processed_count = 0
    skipped_count = 0
    error_details = []

    for i, sample in enumerate(islice(dataset, max_samples)):
        try:
            # Extract text
            text = sample.get('normalized', sample.get('text', '')).strip()

            # Quality filters for text (more lenient)
            if not text or len(text) < min_text_length or len(text) > max_text_length:
                skipped_count += 1
                if i < 10:
                    print(f"Skipped sample {i}: text='{text}', length={len(text)}")
                continue

            # Clean text
            text = clean_hindi_text(text)
            if not text:
                skipped_count += 1
                continue

            # Get audio decoder
            audio_decoder = sample['audio_filepath']

            # Decode using get_all_samples() - returns AudioSamples object
            try:
                audio_samples = audio_decoder.get_all_samples()  # AudioSamples object

                # Access the waveform tensor directly
                audio_tensor = audio_samples.data  # torch.Tensor [num_channels, num_samples]

                # Get sample rate
                original_sr = audio_samples.sample_rate

                print(f"✅ Decoded sample {i}: shape={audio_tensor.shape} at {original_sr}Hz")
            except Exception as e:
                error_details.append(f"Sample {i}: Decoding failed - {str(e)}")
                skipped_count += 1
                if i < 5:
                    print(f"❌ Decoding failed for sample {i}: {str(e)}")
                continue

            # Handle tensor dimensions (ensure [1, time] for mono)
            if audio_tensor.dim() == 1:
                audio_tensor = audio_tensor.unsqueeze(0)
            elif audio_tensor.dim() > 2:
                # Average channels if multi-channel
                audio_tensor = audio_tensor.mean(dim=0, keepdim=True)

            # Skip very short or very long audio
            audio_duration = audio_tensor.size(-1) / original_sr
            if audio_duration < 0.3 or audio_duration > 30.0:
                skipped_count += 1
                if i < 10:
                    print(f"Skipped sample {i}: duration={audio_duration:.2f}s")
                continue

            # Resample if needed
            if original_sr != target_sr:
                resampler = torchaudio.transforms.Resample(
                    orig_freq=original_sr,
                    new_freq=target_sr
                )
                audio_tensor = resampler(audio_tensor)

            # Normalize audio
            if audio_tensor.abs().max() > 0:
                audio_tensor = audio_tensor / audio_tensor.abs().max() * 0.95

            # Save audio file
            wav_filename = f"hindi_{processed_count:06d}.wav"
            wav_path = os.path.join(output_dir, "wavs", wav_filename)
            torchaudio.save(wav_path, audio_tensor, target_sr)

            # Add to metadata
            metadata.append(f"{wav_filename[:-4]}|{text}|{text}")
            processed_count += 1

            # Progress and debug info
            if processed_count <= 5:
                print(f"✅ Sample {processed_count}: '{text[:30]}...', {audio_duration:.2f}s, shape={audio_tensor.shape}")

            if processed_count % 10 == 0:
                print(f"✅ Processed {processed_count}/{max_samples} samples (skipped: {skipped_count})")

        except Exception as e:
            error_msg = f"Sample {i}: {str(e)}"
            error_details.append(error_msg)
            if i < 10:
                print(f"❌ Error processing sample {i}: {str(e)}")
                print(f"Detailed traceback: {traceback.format_exc()}")
            skipped_count += 1

    # Save results
    if processed_count > 0:
        metadata_file = os.path.join(output_dir, "metadata.csv")
        with open(metadata_file, "w", encoding="utf-8") as f:
            f.write("\n".join(metadata))

        print(f"\n🎉 Dataset preparation complete!")
        print(f"📊 Processed: {processed_count} samples")
        print(f"⚠️ Skipped: {skipped_count} samples")
        print(f"💾 Saved to: {output_dir}")

        # Save error report
        if error_details:
            error_file = os.path.join(output_dir, "error_report.txt")
            with open(error_file, "w") as f:
                f.write("\n".join(error_details[:50]))  # Save first 50 errors
    else:
        print(f"\n❌ No samples processed successfully!")
        print(f"⚠️ Skipped: {skipped_count} samples")
        if error_details:
            print("📋 First few errors:")
            for error in error_details[:5]:
                print(f"   {error}")

    return output_dir, processed_count


def clean_hindi_text(text):
    """Enhanced Hindi text cleaning"""
    import re

    if not text:
        return ""

    # Basic cleaning
    text = re.sub(r'\s+', ' ', text).strip()

    # Allow more characters - Hindi, English, numbers, basic punctuation
    # Keep: Devanagari, ASCII, common punctuation
    text = re.sub(r'[^\u0900-\u097F\u0020-\u007E\s।]', '', text)

    # Remove excessive punctuation
    text = re.sub(r'[।।]+', '।', text)
    text = re.sub(r'[\.\.]+', '.', text)

    # Remove parenthetical content (often metadata)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)

    # Final cleanup
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# Test with updated function
if __name__ == "__main__":
    from datasets import load_dataset
    from google.colab import userdata
    from huggingface_hub import login

    print("🧪 Testing updated audio processing...")

    # Login
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)

    # Load dataset
    dataset = load_dataset("ai4bharat/IndicVoices", "hindi", split="train", streaming=True)

    # Test with small number
    result_dir, count = prepare_vits_dataset_fixed(
        dataset,
        output_dir="./test_hindi_data",
        max_samples=10,  # Very small test
        min_text_length=2  # Allow shorter texts for testing
    )

    print(f"\n🎯 Test result: {count} samples processed")

    if count > 0:
        print("🎉 Success! Now you can increase max_samples to 100 or more.")

        # Show what was created
        import os
        if os.path.exists(os.path.join(result_dir, "metadata.csv")):
            with open(os.path.join(result_dir, "metadata.csv"), 'r') as f:
                lines = f.readlines()[:3]
                print("📄 Sample metadata entries:")
                for line in lines:
                    print(f"   {line.strip()}")
    else:
        print("❌ Still failing. Run the debug script first to understand the AudioDecoder.")


🧪 Testing updated audio processing...


Resolving data files:   0%|          | 0/91 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/74 [00:00<?, ?it/s]

🎵 Processing up to 10 samples...
✅ Decoded sample 0: shape=torch.Size([1, 43856]) at 16000Hz
✅ Sample 1: 'जी नमस्ते जी बोलिए...', 2.74s, shape=torch.Size([1, 60440])
✅ Decoded sample 1: shape=torch.Size([1, 4240]) at 16000Hz
Skipped sample 1: duration=0.27s
✅ Decoded sample 2: shape=torch.Size([1, 134640]) at 16000Hz
✅ Sample 2: 'अगर आपका बच्चा चला जाएगा तो फि...', 8.41s, shape=torch.Size([1, 185551])
✅ Decoded sample 3: shape=torch.Size([1, 11552]) at 16000Hz
✅ Sample 3: 'जी बिल्कुल...', 0.72s, shape=torch.Size([1, 15921])


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:337: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.save_with_torchcodec` under the hood. Some parameters like format, encoding, bits_per_sample, buffer_size, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's encoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.encoders.AudioEncoder
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:247: UserWarning: torio.io._streaming_media_encoder.StreamingMediaEncoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s 

✅ Decoded sample 4: shape=torch.Size([1, 73216]) at 16000Hz
✅ Sample 4: 'जी जी वैसे दो ढाई हज़ार लग रहा...', 4.58s, shape=torch.Size([1, 100901])
✅ Decoded sample 5: shape=torch.Size([1, 154000]) at 16000Hz
✅ Sample 5: 'ढाई हज़ार लग रहा है दो तो ठीक ...', 9.62s, shape=torch.Size([1, 212232])
✅ Decoded sample 6: shape=torch.Size([1, 169968]) at 16000Hz
✅ Decoded sample 7: shape=torch.Size([1, 150000]) at 16000Hz
✅ Decoded sample 8: shape=torch.Size([1, 40960]) at 16000Hz
✅ Decoded sample 9: shape=torch.Size([1, 56352]) at 16000Hz

🎉 Dataset preparation complete!
📊 Processed: 9 samples
⚠️ Skipped: 1 samples
💾 Saved to: ./test_hindi_data

🎯 Test result: 9 samples processed
🎉 Success! Now you can increase max_samples to 100 or more.
📄 Sample metadata entries:
   hindi_000000|जी नमस्ते जी बोलिए|जी नमस्ते जी बोलिए
   hindi_000001|अगर आपका बच्चा चला जाएगा तो फिर उनका लॉस हो जाएगा ना बट अगर हम आपको छुट्टी दे भी रहे हैं तो मैम आपका साल ज़्यादा लगेगा|अगर आपका बच्चा चला जाएगा तो फिर उनका लॉस हो ज

## Model Code starts from here



In [48]:
import torch
from torch.utils.data import IterableDataset, DataLoader
from datasets import load_dataset
from itertools import islice
import torchaudio
import re
from huggingface_hub import login
from google.colab import userdata

class StreamingIndicVoicesDataset(IterableDataset):
    def __init__(self, split="train", language="hindi", max_samples=None, target_sr=22050, min_text_len=3, max_text_len=200, min_duration=0.3, max_duration=30.0):
        login(token=userdata.get('HF_TOKEN'))
        self.dataset = load_dataset("ai4bharat/IndicVoices", language, split=split, streaming=True)
        self.max_samples = max_samples
        self.target_sr = target_sr
        self.min_text_len = min_text_len
        self.max_text_len = max_text_len
        self.min_duration = min_duration
        self.max_duration = max_duration

    def __iter__(self):
        iterator = islice(self.dataset, self.max_samples) if self.max_samples else iter(self.dataset)
        for sample in iterator:
            # Process text
            text = sample.get('normalized', sample.get('text', '')).strip()
            if len(text) < self.min_text_len or len(text) > self.max_text_len:
                continue
            text = self.clean_text(text)
            if not text:
                continue

            # Decode audio
            decoder = sample['audio_filepath']
            try:
                audio_samples = decoder.get_all_samples()
                audio_tensor = torch.tensor(audio_samples.data, dtype=torch.float32).unsqueeze(0) if audio_samples.data.dim() == 1 else audio_samples.data
                original_sr = audio_samples.sample_rate
            except:
                continue

            # Duration filter
            duration = audio_tensor.size(-1) / original_sr
            if duration < self.min_duration or duration > self.max_duration:
                continue

            # Resample and normalize
            if original_sr != self.target_sr:
                audio_tensor = torchaudio.transforms.Resample(original_sr, self.target_sr)(audio_tensor)
            if audio_tensor.abs().max() > 0:
                audio_tensor = audio_tensor / audio_tensor.abs().max() * 0.95

            yield {'audio': audio_tensor, 'text': text, 'duration': duration}

    def clean_text(self, text):
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[^\u0900-\u097F\u0020-\u007E\s।]', '', text)
        text = re.sub(r'[।।]+', '।', text)
        text = re.sub(r'[\.\.]+', '.', text)
        text = re.sub(r'\([^)]*\)', '', text)
        text = re.sub(r'\[[^\]]*\]', '', text)
        return text.strip() or None

# Custom collate for variable-length batches
def vits_collate(batch):
    audios = [item['audio'] for item in batch]
    texts = [item['text'] for item in batch]
    max_len = max(a.size(-1) for a in audios)
    padded_audios = torch.stack([torch.nn.functional.pad(a, (0, max_len - a.size(-1))) for a in audios])
    return {'audio': padded_audios, 'text': texts}


In [21]:
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/monotonic_align
!python setup.py build_ext --inplace
%cd ../..

Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 7.19 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance

In [64]:
# Remove any existing vits directory to avoid conflicts
!rm -rf /content/vits

# Re-clone the VITS repository
!git clone https://github.com/jaywalnut310/vits.git

# Navigate and build monotonic_align
%cd /content/vits/monotonic_align

# Create the required nested directory
!mkdir -p monotonic_align

# Copy core.pyx into the nested directory (this assumes the clone succeeded)
!cp core.pyx monotonic_align/

# Build the extension
!python setup.py build_ext --inplace

# Verify the .so file was created
!ls -l monotonic_align/

%cd /content


Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 6.47 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance

In [85]:
# Install dependencies
!pip install torch torchaudio torchcodec>=0.6.1 datasets huggingface_hub transformers tensorboard librosa soundfile flask flask-socketio websockets cython numpy scipy

# Remove any existing vits directory
!rm -rf /content/vits

# Clone VITS repository
!git clone https://github.com/jaywalnut310/vits.git

# Fix monotonic_align build
%cd /content/vits/monotonic_align
!rm -rf build monotonic_align *.so *.c *.o  # Clean artifacts
!mkdir -p monotonic_align/monotonic_align  # Nested directories
!cp core.pyx monotonic_align/monotonic_align/
!python setup.py build_ext --inplace

# Verify build
!ls -l monotonic_align/monotonic_align  # Should show core.cpython-312-x86_64-linux-gnu.so

%cd /content


Cloning into 'vits'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (81/81), 3.33 MiB | 7.29 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/vits/monotonic_align
Compiling core.pyx because it changed.
[1/1] Cythonizing core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance